In [1]:
# 00_build_event_list  ──────────────────────────────────────────────────────
#
# Inputs
#   ../data_raw/ceo_events.csv      (10k-row raw dismissal file you uploaded)
#
# Outputs
#   ../data_raw/ceo_events_big.csv  (100 events: company, ticker, event_date)
#
# How it works
#   1. Parse the correct date column (leftofc or dismissal_date).
#   2. Keep exits from 2010-2024.
#   3. If the raw set already has a usable 'ticker' field, use it.
#      Otherwise, look up tickers with yfinance (early-exit after 100 hits).
#   4. One event per ticker → latest exit.
#   5. Save and show a preview.

import os, re, sys
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm

# ── Config ────────────────────────────────────────────────────────────────
RAW_PATH   = Path("../data_raw/ceo_events.csv")   # relative to notebooks/
OUT_PATH   = Path("../data_raw/ceo_events_big.csv")
TARGET_CT  = 100
YEAR_START = 2010
YEAR_END   = 2024

# ── 1. Load raw dataset ───────────────────────────────────────────────────
if not RAW_PATH.exists():
    sys.exit(f"✖ Raw file not found: {RAW_PATH.resolve()}")

raw = pd.read_csv(RAW_PATH)
print(f"Loaded {len(raw):,} rows from {RAW_PATH.name}")

# ── 2. Build a proper datetime column called 'event_dt' ───────────────────
date_col = None
for cand in ("leftofc", "dismissal_date", "leftofc_dt"):
    if cand in raw.columns:
        date_col = cand
        break
if date_col is None:
    sys.exit("✖ No dismissal date column found (looked for leftofc / dismissal_date)")

raw["event_dt"] = pd.to_datetime(raw[date_col], errors="coerce")
raw = raw.dropna(subset=["event_dt"]).query(
        f"{YEAR_START} <= event_dt.dt.year <= {YEAR_END}"
)
print(f"→ {len(raw):,} rows after date filter {YEAR_START}–{YEAR_END}")

# ── 3. Use ticker column if it exists and is non-null ─────────────────────
if "ticker" in raw.columns and raw["ticker"].notna().sum() > 0:
    use_ticker = True
    print("✓ Using ticker symbols from the raw file")
    events = (
        raw.dropna(subset=["ticker"])
            .sort_values("event_dt")
            .groupby("ticker", as_index=False)
            .last()                               # keep latest exit per ticker
            .rename(columns={"coname": "company",
                             "event_dt": "event_date"})
            [["company", "ticker", "event_date"]]
            .head(TARGET_CT)
    )
else:
    use_ticker = False
    print("⚠ No ticker column found – falling back to yfinance lookup")
    try:
        from yfinance import Ticker
    except ImportError as e:
        sys.exit("Install yfinance first:  pip install yfinance")

    # de-duplicate by company name (keep latest exit)
    dedup = (
        raw.sort_values("event_dt")
            .groupby("coname", as_index=False)
            .last()[["coname", "event_dt"]]
    )

    def clean_name(name: str) -> str:
        """Drop suffixes like INC, CORP for better yfinance matches."""
        return re.sub(r"\b(INC|CORP|CO|INC\.|CORPORATION|LTD|PLC)\b", "",
                      name, flags=re.I).strip()

    def get_ticker(name: str) -> str | None:
        try:
            info = Ticker(clean_name(name)).get_info()
            tkr = info.get("symbol")
            return tkr if tkr and 1 <= len(tkr) <= 5 else None
        except Exception:
            return None

    # build events list with early exit after TARGET_CT successes
    rows = []
    for _, row in tqdm(dedup.iterrows(), total=len(dedup), desc="yfinance"):
        tkr = get_ticker(row.coname)
        if tkr:
            rows.append({"company": row.coname,
                         "ticker":  tkr,
                         "event_date": row.event_dt})
            if len(rows) == TARGET_CT:
                break

    events = pd.DataFrame(rows)
    if len(events) < TARGET_CT:
        print(f"⚠ Only found {len(events)} valid tickers; proceed anyway.")

# ── 4. Save & preview ─────────────────────────────────────────────────────
OUT_PATH.parent.mkdir(parents=True, exist_ok=True)
events.to_csv(OUT_PATH, index=False)

print(f"\n✓ Wrote {len(events)} rows → {OUT_PATH.resolve()}")
display(events.head())


c:\Users\Ashis\miniconda3\envs\ceonlp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loaded 9,390 rows from ceo_events.csv
→ 2,888 rows after date filter 2010–2024
⚠ No ticker column found – falling back to yfinance lookup


yfinance: 100%|██████████| 1951/1951 [34:46<00:00,  1.07s/it]

⚠ Only found 78 valid tickers; proceed anyway.

✓ Wrote 78 rows → C:\Users\Ashis\Downloads\ceo_sentiment\data_raw\ceo_events_big.csv


,company,ticker,event_date
0,AAON INC,AAON,2020-05-12
1,AAR CORP,AAR,2018-05-31
2,ADT CORP,ADT,2016-05-02
3,AES CORP,AES,2011-09-30
4,AMGEN INC,AMGEN,2012-05-23
